In [1]:
import os
import sys
import pickle
import numpy as np
import pandas as pd
import seaborn as sns
from copy import deepcopy
from matplotlib import pyplot as plt
from IPython.display import clear_output
from matplotlib.colors import TABLEAU_COLORS

sys.path.append('..')
from notebook_utils import Data, train, run_study, test_vanilla, test_uncertain

sys.path.append('../../..')
from uncertain.pairwise_ranking.MF import bprMF, UncertainMF, PretrainedUncertainMF
from uncertain.extras import Resample

if os.path.isfile('data.pkl'):
    with open('data.pkl', 'rb') as f:
        data = pickle.load(f)
    print(f'Data prepared: {data.n_user} users, {data.n_item} items.')
    print(f'{len(data.train)} train, {len(data.val)} validation and {len(data.test)} test interactions.')
    data.batch_size = int(1e3)
else:
    data = pd.read_table('100k_a.csv', sep=',', header=None)
    data = data[[0, 1]].rename(columns={0: 'user', 1: 'item'})
    
    # Drop items with < 5 ratings
    length = data.item.value_counts()
    data.drop(data.index[data.item.isin(length.index[length < 5])], 0, inplace=True)
    
    # Drop user with < 20 ratings
    length = data.user.value_counts().drop(columns='timestamps')
    data.drop(data.index[data.user.isin(length.index[length < 5])], 0, inplace=True)
    
    data = Data(data)
    with open('data.pkl', 'wb') as f:
        pickle.dump(data, f, protocol=5)
        
base_conf = {'embedding_dim':1, 'lr':0, 'weight_decay':0}
trials = 1 ## 0 for eval only mode

Data prepared: 80103 users, 97306 items.
1294893 train, 392631 validation and 420192 test interactions.


In [2]:
f'batch size: ({2**8}, {2**10}) - learning rate: ({1e-5}, {1e-3} - weight_decay: {1e-5, 1e-3})'

'batch size: (256, 1024) - learning rate: (1e-05, 0.001 - weight_decay: (1e-05, 0.001))'

In [3]:
name = 'MF-BPR'
def init_model(**kwargs):
    return bprMF(data.n_user, data.n_item, **kwargs)

def objective(trial):
    
    # Parameter setup
    batch_size = trial.suggest_int('bs', 2**8, 2**10)
    params = {'embedding_dim': trial.suggest_int('dim', 200, 200),
              'lr': trial.suggest_float('lr', 1e-5, 1e-3),
              'weight_decay': trial.suggest_float('wd', 1e-5, 1e-3)}
    '''
    'weight_decay': (trial.suggest_float('wd_user', 1e-4, 1e-2), 
                     trial.suggest_float('wd_item', 1e-4, 1e-2), 
                     trial.suggest_float('wd_neg_item', 1e-4, 1e-2))
    '''
    params_str = f'bs = {batch_size}-' + '-'.join(f'{key}={value}' for key, value in params.items())

    # Train
    data.batch_size = batch_size
    model = init_model(**params)
    MAP, path = train(model, data, path='checkpoints/' + name, name=params_str)
    trial.set_user_attr('filename', path)
    return MAP

study = run_study(name, objective, n_trials=2)
best_runs = study.trials_dataframe().sort_values('value')[::-1][:5]
baseline = init_model(**base_conf).load_from_checkpoint(best_runs.user_attrs_filename.iloc[0])
results = test_vanilla(baseline, data, max_k=10, name=name)

clear_output(wait=True)
print(best_runs.user_attrs_filename.iloc[0])
print(results)
best_runs

/home/vcoscrato/Documents/RecSys/MF/tests/Pairwise/Twitch/checkpoints/MF-BPR/bs = 514-embedding_dim=200-lr=6.761435842150177e-05-weight_decay=0.00019672288025825879-epoch=179-val_likelihood=0.9615027904510498.ckpt
{'FCP': 0.9736834589901759, 'MAP': array([0.03228344, 0.04537283, 0.0534    , 0.05851945, 0.06222538,
       0.06491446, 0.067158  , 0.06883249, 0.07017951, 0.07138138],
      dtype=float32), 'Recall': array([0.01009384, 0.0184634 , 0.02633828, 0.03336545, 0.03995734,
       0.04583093, 0.05203761, 0.0571884 , 0.06269204, 0.06844836],
      dtype=float32)}


,number,value,datetime_start,datetime_complete,duration,params_bs,params_dim,params_lr,params_wd,user_attrs_filename,state
0,0,0.961460,2022-07-05 01:21:09.644485,2022-07-05 07:34:37.256720,0 days 06:13:27.612235,514,200,0.000068,0.000197,/home/vcoscrato/Documents/RecSys/MF/tests/Pair...,COMPLETE
1,1,0.954646,2022-07-05 07:34:37.257892,2022-07-05 10:16:51.765993,0 days 02:42:14.508101,388,200,0.000595,0.000944,/home/vcoscrato/Documents/RecSys/MF/tests/Pair...,COMPLETE


In [ ]:
name = 'UMF-GPR-notailchasing'

def objective(trial):
    
    # Parameter setup
    batch_size = trial.suggest_int('bs', 2**8, 2**10)
    params = {'embedding_dim': trial.suggest_int('dim', 200, 200),
              'embedding_dim_var': trial.suggest_int('dim_var', 5, 50),
              'lr': trial.suggest_float('lr', 1e-5, 1e-3),
              'weight_decay': trial.suggest_float('wd', 1e-6, 1e-4)}
    params_str = f'bs = {batch_size}-' + '-'.join(f'{key}={value}' for key, value in params.items())

    # Train
    data.batch_size = batch_size
    model = UncertainMF(data.n_user, data.n_item, **params)
    
    # Freeze uncertainty
    for param in model.user_var.parameters():
        param.requires_grad = False
    for param in model.item_var.parameters():
        param.requires_grad = False
     
    _, path = train(model, data, path='checkpoints/' + name, name=params_str)
    model.load_from_checkpoint(path)
    
    # Unfreeze uncertainty
    for param in model.user_var.parameters():
        param.requires_grad = True
    for param in model.item_var.parameters():
        param.requires_grad = True
        
    # Freeze mean
    for param in model.user_mean.parameters():
        param.requires_grad = False
    for param in model.item_mean.parameters():
        param.requires_grad = False
    
    MAP, path = train(model, data, path='checkpoints/' + name, name=params_str)
    trial.set_user_attr('filename', path)
    
    return MAP

study = run_study(name, objective, n_trials=5)
best_runs = study.trials_dataframe().sort_values('value')[::-1][:5]
model = UncertainMF(data.n_user, data.n_item, 1, 1, 1, 1).load_from_checkpoint(best_runs.user_attrs_filename.iloc[0])
results = test_uncertain(model, data, max_k=10, name=name)

# clear_output(wait=True)
print(best_runs.user_attrs_filename.iloc[0])
print(results)
best_runs

[I 2022-07-05 10:46:10,171] A new study created in memory with name: no-name-6c886914-c4c1-4e19-875e-0defec3fc0ae
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
/home/vcoscrato/.local/lib/python3.8/site-packages/pytorch_lightning/core/datamodule.py:469: LightningDeprecationWarning: DataModule.setup has already been called, so it will not be called again. In v1.6 this behavior will change to always call DataModule.setup.
  rank_zero_deprecation(
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name           | Type      | Params
---------------------------------------------
0 | user_mean      | Embedding | 16.0 M
1 | item_mean      | Embedding | 19.5 M
2 | user_var       | Embedding | 3.3 M 
3 | item_var       | Embedding | 4.0 M 
4 | var_activation | Softplus  | 0     
---------------------------------------------
35.5 M    Trainable params
7.3 M     Non-trainable params
42.8 M    Total params
171.022   Total estimated model p

Validation sanity check: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7fcc26bc6c10>
Traceback (most recent call last):
  File "/home/vcoscrato/.local/lib/python3.8/site-packages/torch/utils/data/dataloader.py", line 1328, in __del__
    self._shutdown_workers()
  File "/home/vcoscrato/.local/lib/python3.8/site-packages/torch/utils/data/dataloader.py", line 1320, in _shutdown_workers
    if w.is_alive():
  File "/usr/lib/python3.8/multiprocessing/process.py", line 160, in is_alive
    assert self._parent_pid == os.getpid(), 'can only test a child process'
AssertionError: can only test a child process
Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7fcc26bc6c10>
Traceback (most recent call last):
  File "/home/vcoscrato/.local/lib/python3.8/site-packages/torch/utils/data/dataloader.py", line 1328, in __del__
    self._shutdown_workers()
  File "/home/vcoscrato/.local/lib/python3.8/site-packages/torch/utils/data/dataloader.py", line 1320, in _shutdown_wor